In [2]:
import pandas as pd 
import os 
import sys 
from pycaret.classification import *

sys.path.append('../src')
from utils import getExperimentConfig, getPicklesFromDir

import pickle

In [3]:
config = getExperimentConfig()
folders=config['folders']
dataset_settings = getPicklesFromDir(folders['settings_dir'])
display(config)

{'folders': {'log_dir': '../logs/',
  'data_dir': '../data/',
  'sd_dir': '../data/synthetic/',
  'real_dir': '../data/real/',
  'train_dir': '../data/train/',
  'test_dir': '../data/test/',
  'pickles_dir': '../pickles/',
  'models_dir': '../pickles/models/',
  'SDGs_dir': '../pickles/SDGs/',
  'settings_dir': '../pickles/settings/',
  'setup_dir': '../pickles/pycaret_setup/'},
 'ml_models': ['lr', 'knn', 'nb', 'svm', 'rbfsvm', 'rf']}

In [4]:
df = pd.read_csv(folders['sd_dir']+"SD0Q1_0.csv")

In [5]:
df.head()

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,1,114,89,2,214,37.5,0.364,48,0
1,1,4,171,41,37,140,20.6,0.335,21,0
2,2,0,110,60,28,434,26.7,0.270,26,0
3,3,0,80,82,18,0,22.1,0.204,41,0
4,4,1,92,74,21,2,23.6,0.198,25,0


In [6]:
#update setup_param log setings
dataset_settings[0]['setup_param']['experiment_name'] = "SD0Q1_0"
dataset_settings[0]['setup_param']['system_log'] = folders['log_dir']+"SD0Q1_0"
# Do not save train test datasplit, to save resources
dataset_settings[0]['setup_param']['log_data'] = False


pycaret_setup = setup(data=pd.read_csv(folders['sd_dir']+"SD0q1_0.csv"), 
                      **dataset_settings[0]['setup_param'])

,Description,Value
0,Session id,6025
1,Target,Outcome
2,Target type,Binary
3,Original data shape,"(768, 10)"
4,Transformed data shape,"(768, 10)"
5,Transformed train set shape,"(614, 10)"
6,Transformed test set shape,"(154, 10)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


In [7]:
# the dataset specific saved experiment
#config_filename = folders['setup_dir']+'D0.pkl'

# load the experiment but with the synthetic data, and test_data=original_data.csv
#load_experiment(config_filename, data=pd.read_csv(folders['sd_dir']+"SD0Q1_0.csv"))
# Update log experiment name
#set_config('exp_name_log', 'SD0Q1_0')

In [8]:
log = get_logs(exp_name)

In [11]:
log.columns

Index(['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time',
       'end_time', 'metrics.Recall', 'metrics.MCC', 'metrics.Kappa',
       'metrics.F1',
       ...
       'tags.mlflow.source.type', 'tags.URI', 'tags.Source', 'tags.Run ID',
       'tags.mlflow.user', 'tags.Run Time', 'tags.mlflow.source.name',
       'tags.USI', 'tags.mlflow.log-model.history', 'tags.mlflow.parentRunId'],
      dtype='object', length=103)

In [ ]:

best = compare_models(include=config['ml_models'])

In [ ]:
# Dataset of performance with
tune_model(best, optimize='Accuracy', choose_better=True, n_iter=200)


In [14]:
model = load_model(folders['models_dir']+'D0')

Transformation Pipeline and Model Successfully Loaded


In [25]:
exp_name =dataset_settings[0]['meta']['id']
#Get logs
get_logs(exp_name)[[
    'tags.mlflow.runName',
    'metrics.Accuracy',
    'metrics.F1',
    'metrics.Prec',
    'metrics.Recall',
    'tags.Source',
    'run_id', 
]].sort_values('metrics.Accuracy', ascending=False)

,tags.mlflow.runName,metrics.Accuracy,metrics.F1,metrics.Prec,metrics.Recall,tags.Source,run_id
7,Random Forest Classifier,0.7900,0.6711,0.7465,0.6167,compare_models,a5d9b18b84d04269b0d11d3df3dde699
6,Logistic Regression,0.7754,0.6428,0.7319,0.5840,compare_models,8790ff71a4374e30a6297cd62365584c
1,Random Forest Classifier,0.7671,0.6981,0.6416,0.7749,tune_model,79492964faa146c8aa95bb2bfd4a6e6c
5,SVM - Radial Kernel,0.7655,0.6296,0.7207,0.5742,compare_models,656021909f5f48de9dcba758c3065bb0
4,Naive Bayes,0.7639,0.6462,0.6890,0.6203,compare_models,a1671c530fe144cea6b708ae50acaac6
3,K Neighbors Classifier,0.7558,0.6142,0.6864,0.5701,compare_models,c407633800fd46229a4cf6118f1334c7
2,SVM - Linear Kernel,0.7279,0.5689,0.6376,0.5238,compare_models,9e9d07ca556b452eb05165f67e599b98
0,Session Initialized 0dfa,NaN,NaN,NaN,NaN,setup,0ddcd068a0a24037971bfa78547b4270
8,Session Initialized 09bf,NaN,NaN,NaN,NaN,setup,8865811c66f74657b50760513f858724
